In [1]:
from scripts import setup_environment

setup_environment()

In [2]:
from scripts.dataset import BrainteaserDataset

dataset = BrainteaserDataset("data")

In [3]:
import string

from scripts.dataset import RiddleQuestion
from scripts.prompt_helpers import create_prompt_template


def args_generator(riddle_question: RiddleQuestion):
    template_args = {
        "question": riddle_question.question,
        "choices": "\n".join(
            [
                f"({string.ascii_uppercase[j]}) {choice}"
                for j, choice in enumerate(riddle_question.choice_list)
            ]
        ),
        "answer": string.ascii_uppercase[riddle_question.label],
    }

    return template_args


chat_prompt_template = create_prompt_template("default")

In [4]:
from scripts.lmm import OllamaModelBuilder
from scripts.executor import Executor

base_url = "http://104.15.30.249:46506"
model_builder = OllamaModelBuilder(base_url)

executor = Executor(
    models=[
        # Llama3.1
        model_builder.build_model("llama3.1:8b-instruct-q8_0"),  # => 9 GB
        # Llama3.2
        model_builder.build_model("llama3.2:1b-instruct-fp16"),  # => 2.5 GB
        model_builder.build_model("llama3.2:3b-instruct-fp16"),  # => 6.4 GB
        # Phi3.5
        model_builder.build_model("phi3.5:3.8b-mini-instruct-fp16"),  # => 7.6 GB
        # Phi4
        model_builder.build_model("phi4:14b-q8_0"),  # => 16 GB
        # Qwen2.5
        model_builder.build_model("qwen2.5:0.5b-instruct-fp16"),  # => 1 GB
        model_builder.build_model("qwen2.5:1.5b-instruct-fp16"),  # => 3.1 GB
        model_builder.build_model("qwen2.5:3b-instruct-fp16"),  # => 6.2 GB
        model_builder.build_model("qwen2.5:7b-instruct-q8_0"),  # => 8.1 GB
        model_builder.build_model("qwen2.5:14b-instruct-q8_0"),  # => 16 GB
        model_builder.build_model("qwen2.5:32b-instruct-q4_K_M"),  # => 20 GB
        # Gemma2
        model_builder.build_model("gemma2:2b-instruct-fp16"),  # => 5.2 GB
        model_builder.build_model("gemma2:9b-instruct-q8_0"),  # => 9.8 GB
        model_builder.build_model("gemma2:27b-instruct-q4_K_M"),  # => 22 GB
        # Mistral Nemo
        model_builder.build_model("mistral-nemo:12b-instruct-2407-q8_0"),  # => 13 GB
    ]
)

2025-03-11 22:52:55,678 - INFO - Initialized executor with 15 models.


In [5]:
import numpy as np

from scripts.executor import Dataset

# Set fixed seed for reproducibility
np.random.seed(42)

maximal_n = 8


def create_n_shot_dataset(
    data: list[RiddleQuestion],
    name: str,
    example_count: int = 10,
) -> tuple[list[RiddleQuestion], Dataset]:
    """
    Create a few-shot learning dataset by selecting diverse examples and the remaining data for testing.

    Args:
        data: List of riddle questions
        name: Name of the dataset
        example_count: Number of examples to use for few-shot learning

    Returns:
        tuple: (examples for few-shot learning, remaining dataset for testing)
    """
    # Group data by answer choice
    answer_groups = {}
    for i, question in enumerate(data):
        answer = question.label
        if answer not in answer_groups:
            answer_groups[answer] = []
        answer_groups[answer].append(i)

    # Select diverse examples for few-shot learning
    example_indices = []
    answers = list(answer_groups.keys())

    # Distribute examples evenly across answer choices
    while len(example_indices) < example_count and answers:
        for answer in list(answers):  # Use a copy to safely modify during iteration
            if answer_groups[answer]:
                example_indices.append(answer_groups[answer].pop(0))
                if len(example_indices) >= example_count:
                    break
            else:
                answers.remove(answer)

    # If we still need more examples, take randomly from remaining data
    if len(example_indices) < example_count:
        remaining_indices = [i for i in range(len(data)) if i not in example_indices]
        np.random.shuffle(remaining_indices)
        example_indices.extend(
            remaining_indices[: example_count - len(example_indices)]
        )

    # Get the examples
    examples = [data[i] for i in sorted(example_indices[:example_count])]

    # Create dataset from all remaining data (not used as examples)
    remaining_indices = [i for i in range(len(data)) if i not in example_indices]
    test_dataset = Dataset(name=name, riddles=[data[i] for i in remaining_indices])

    return examples, test_dataset


# Create test datasets
sp_examples, sp_data = create_n_shot_dataset(dataset.sp, "sp", example_count=maximal_n)
wp_examples, wp_data = create_n_shot_dataset(dataset.wp, "wp", example_count=maximal_n)

# Prepare executor data
executor_data = [sp_data, wp_data]

### Few Shot Helpers


In [6]:
from collections.abc import Callable

import dill as pickle
from langchain_core.prompts import ChatPromptTemplate

from scripts.prompt_helpers import TemplateNameType, get_few_shot_chat_template

# Get the best prompt type for each model
with open("results/best_n_value_by_model.pkl", "rb") as f:
    best_n_value_by_model = pickle.load(f)


def few_shot_prompt_template_generator(
    model_name: str, dataset: Dataset
) -> Callable[[str], ChatPromptTemplate]:
    if dataset.name == "sp":
        few_shot_examples = sp_examples
    elif dataset.name == "wp":
        few_shot_examples = wp_examples
    else:
        raise ValueError(f"Unknown dataset: {dataset.name}")

    model_name = model_name[0 : model_name.index("b-") + 1]
    best_system_template_name: TemplateNameType = "default"
    number_of_shots = best_n_value_by_model[model_name][dataset.name]
    template = get_few_shot_chat_template(
        few_shot_examples,
        args_generator,
        best_system_template_name,
        number_of_shots,
    )
    return template

In [7]:
results = await executor.aexecute(
    executor_data,
    lambda model_name, dataset: few_shot_prompt_template_generator(model_name, dataset),
    args_generator,
    dump_to_pickle=True,
    create_checkpoints=True,
    resume_from_checkpoint=True,
    run_name="baseline_few-shot_evaluation",
)

2025-03-11 22:52:55,744 - INFO - Starting execution 'baseline-few-shot-evaluation': 2 dataset(s) x 15 model(s) = 16545 riddle evaluations


baseline-few-shot-evaluation:   0%|          | 0/16545 [00:00<?, ?it/s]

2025-03-12 00:22:56,074 - INFO - Saving results to results/baseline-few-shot-evaluation/baseline-few-shot-evaluation_results.pkl
2025-03-12 00:23:08,597 - INFO - Execution 'baseline-few-shot-evaluation' completed successfully.
